# レコメンドエンジンのお勉強
---

## お勉強するモデル と 目的

### そもそもレコメンドとは

- あるコンテンツ、映画や商品をユーザーに向けておすすめすること
- レコメンドエンジンでは、おすすめの精度、即ち、CVRやCVをより高くすることを目指す。

各モデルの実装をすることで、仕組みを知った上で使うことを目指す。

- 協調フィルタリング
- MatrixFactorization　（MF）
- FactorizationMachines　（FM）
- DeepFM
- Recommendation_with_Graph


## データセット

- MovieLens 100K Dataset
    - https://grouplens.org/datasets/movielens/100k/
- `../ml-100k/u.data`を使用する。
    - user_id, item_id, rating, timestamp の 4カラムのtsv


## 協調フィルタリング

- ユーザーベース・アイテムベースなどの手法がある
- 例えば、ユーザー同士の類似度を算出し、対象のユーザーがまだ消費していない and おすすめ度の高いコンテンツをレコメンドする。
- ユーザーの類似度を算出方法には、多次元のベクトル同士がどれだけ似ているかがわかればよい
    - 今回は、コサイン類似度を用いる
    
### 抱えている問題
- スパースなデータセットの場合、類似ユーザーを見つけにくい
- 誰も評価していないアイテムはレコメンドされない

In [1]:
import numpy as np
import pandas as pd

from scipy.spatial import distance

In [52]:
col_names = ['user_id', 'item_id', 'rating', 'timestamp']

udata_df = pd.read_csv('../ml-100k/u.data', names=col_names, sep='\t')

In [53]:
print (udata_df.shape)
udata_df.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [57]:
df = udata_df.pivot_table(index='user_id', columns='item_id', values='rating')

In [58]:
print (df.shape)
df.head()

(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
user_0 = df.iloc[0].fillna(0)
user_1 = df.iloc[1].fillna(0)
user_2 = df.iloc[2].fillna(0)

print ('user_0 と user_1の類似度：', distance.cosine(user_0, user_1))
print ('user_0 と user_2の類似度：', distance.cosine(user_0, user_2))

user_0 と user_1の類似度： 0.833069016131298
user_0 と user_2の類似度： 0.9525404571744682


user_0 は user_1 よりも user_2 の方が類似度が高いと言える.
  
user_0 には まだ評価をしていない and user_2が最も高評価している映画をレコメンドすればよい

In [92]:
trans_df = df.iloc[:2].T

candidate_df = trans_df.loc[(trans_df[1].isnull() & ~trans_df[2].isnull()), :]
candidate_df = candidate_df.sort_values(by=2, ascending=False)

print ('おすすめのitem_idは：', candidate_df.head().index.values)
candidate_df.head()

おすすめのitem_idは： [316 275 313 311 285]


user_id,1,2
item_id,,
316,NaN,5.0
275,NaN,5.0
313,NaN,5.0
311,NaN,5.0
285,NaN,5.0


## MatrixFactorization （MF）

- user * item のマトリックスを2つの任意のk次元ベクトルに分解する
    - userベクトル -> user数 * k
    - itemベクトル -> k * item数
- 各ベクトルのk次元への圧縮はSVDで行う
- 各ベクトルの内積をとることによって、予測評価値を算出する
- 実際の予測評価値によせて、fittingを行う
- 欠損値は、基本的に0埋めする

### 課題

- 全体の損失をRMSEで評価を行った
- 圧縮する次元Kを増やすと精度が上がることがわかった。
- だが、Kを増やすと計算量が増えるため、適切なKを決定することが難しい

In [93]:
import numpy as np
import pandas as pd

In [97]:
col_names = ['user_id', 'item_id', 'rating', 'timestamp']
udata_df = pd.read_csv('../ml-100k/u.data', names=col_names, sep='\t')

df = udata_df.pivot_table(index='user_id', columns='item_id', values='rating')

print (df.shape)
df.head()

(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 調整するパラメータは初期値を割り当てたk次元のユーザーベクトルとアイテムベクトル